<a href="https://colab.research.google.com/github/Yibei990826/BestBuy_Sales_forecast/blob/main/evaluation_metrics_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install SentencePiece datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/dlt_results/

/content/drive/MyDrive/dlt_results


In [4]:
import re
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import transformers
from torch.optim import Adam
from tqdm import tqdm
from utils import *
import json
# from datasets import Dataset, load_dataset

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

In [6]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

## Dataset processing

In [9]:
with open("./dev.json") as input_file:
      valid_data = json.load(input_file)

def ad_data(json_data, verbose=True):
    net=[]
    err_cnt=0
    for example in json_data:
        question = example["qa"]["question"]
        table = example["table"]
        table_text = ""
        for row in table[1:]:
            this_sent = table_row_to_text(table[0], row)
            table_text += this_sent
        try:
            steps_text = format_steps(example["qa"]["steps"])
            inputs = {"context": table_text, "question": question, "answer": steps_text, "id":example["id"]}
            net.append(inputs)
        except:
            err_cnt+=1
            if verbose:
                print ("-"*25)
                print (example["filename"])
                print (example["qa"]["steps"])
                print ("-"*25+"\n")
    if err_cnt>0:
        print ("Net Errors:",err_cnt)
    return net

valid_pd = pd.DataFrame(ad_data(valid_data,False))

Net Errors: 35


In [10]:
step_pattern = r"Step \d+"
import re
# def num_steps_dict_fn(example):
#     example["num_steps"] = len(re.findall(step_pattern, example["answer"]))
#     return example

In [11]:
# data_splits = {'dev': valid_data}
valid_pd = pd.DataFrame(ad_data(valid_data,False))
valid_pd["num_steps"] = valid_pd["answer"].apply(lambda x: len(re.findall(step_pattern, x)))
# dataset_hf = {"valid": Dataset.from_pandas(valid_pd)}

Net Errors: 35


In [ ]:
# dataset_hf['valid']=dataset_hf['valid'].map(num_steps_dict_fn)

In [ ]:
# dataset_hf_single = dataset_hf['valid'].filter(lambda example: example['num_steps']==1)

In [ ]:
# from torch.utils.data import Dataset as Dataset_torch

In [12]:
TOKENIZER = T5Tokenizer.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("./finqa_finetune_t5.pth/",from_tf=False)
BATCH_SIZE = 8
TESTING = False
MODEL.to(DEVICE);

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [13]:
from utils import FinQA_Dataset

In [14]:
MAX_Q_LEN = 1024
MAX_A_LEN = 128

finqa_valid1 = FinQA_Dataset(TOKENIZER, valid_pd, MAX_Q_LEN, MAX_A_LEN)
finqa_valid1

In [15]:
len(finqa_valid1)

848

# Inference

## Inefernce on 1 step validation data

In [ ]:
# from torch.utils.data import DataLoader as DataLoader_torch

In [16]:
test_loader = DataLoader(finqa_valid1, batch_size=4)

In [95]:
MODEL.eval();
label_texts = []
generated_texts = []
for batch in tqdm(test_loader, desc="Generation batches"):
    input_ids = batch["input_ids"].to(DEVICE)
    attention_mask = batch["attention_mask"].to(DEVICE)
    decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)
    with torch.no_grad():
        # Use the `generate` method for text generation
        generated_output = MODEL.generate(
            input_ids=input_ids,
            max_length=128,  # Set the desired maximum length for generated text
            num_beams=5,    # You can adjust the number of beams for beam search
            temperature=0.8  # You can adjust the temperature for sampling,
            )
        for i,output_i in enumerate(generated_output):
          generated_text = TOKENIZER.decode([x for x in output_i if x!= -100], skip_special_tokens=True)
          generated_texts.append(generated_text)
          label_text = TOKENIZER.decode([x for x in batch["labels"][i] if x!= -100], skip_special_tokens=True)
          label_texts.append(label_text)

Generation batches:   0%|          | 0/212 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Generation batches:   4%|▍         | 9/212 [00:18<06:53,  2.03s/it]


KeyboardInterrupt: ignored

In [96]:
result_df = pd.DataFrame({"generated_text":generated_texts, "label_text":label_texts})

## Postprocessing

In [90]:
def extract_info(input_text):
  try:
      # print ("inside try")
      # Regular expression to match the answer pattern
      answer_pattern1 = r"Step 1: ([A-Za-z]+) ([-]?[$]?[\d,.]+) .* ([-]?[$]?[\d,.]+)\.* This gives the result: ([-]?[$]?[\d.]+%?)"
      answer_pattern2 = r"Step 1: ([A-Za-z]+) ([-]?[$]?[\w.,]+) .* ([-]?[$]?[\w.,]+)\.* This gives the result: ([-]?[$]?\w+)"

      # Find the matching groups
      match1 = re.search(answer_pattern1, input_text)
      match2 = re.search(answer_pattern2, input_text)
      if match1:
          operation, arg1, arg2, result = match1.groups()
          # Removing commas from the arguments and converting to appropriate types
          chars_to_replace=[',','$']
          for char in chars_to_replace:
              arg1=arg1.replace(char,'')
              arg2=arg2.replace(char,'')
              result=result.replace(char,'')
          # arg1 = float(arg1)
          # arg2 = float(arg2)
          # Checking if result is a percentage or a number
          if '%' in result:
              result = (float(result.replace('%', ''))/100)
          else:
              result = (result)
          # Constructing the output dictionary
          output = {'Answer': match1.group(0), 'Operation': operation.capitalize(), 'argument_1': arg1,
                    'argument_2': arg2,'result': result}
          return output
      if match2:
          operation, arg1, arg2, result = match2.groups()
          # Removing commas from the arguments and converting to appropriate types
          chars_to_replace=[',','$']
          for char in chars_to_replace:
              arg1=arg1.replace(char,'')
              arg2=arg2.replace(char,'')
              result=result.replace(char,'')
          # arg1 = float(arg1)
          # arg2 = float(arg2)
          # Checking if result is a percentage or a number
          if '%' in result:
              result = (float(result.replace('%', ''))/100)
          else:
              result = (result)
          # Constructing the output dictionary
          output = {'Answer': match1.group(0), 'Operation': operation.capitalize(), 'argument_1': arg1,
                    'argument_2': arg2,'result': result}
          return output
  except:
      print ("inside except")
      return "FUCKK"

In [97]:
result_df["label_text"][0]

'Step 1: Divide 637 by const_5. This gives the result: 127.40'

In [132]:
def simplify(x):
    if ',' in x:
        x = x.replace(',', '')  # Remove commas
    if 'const_' in x:
        x = x.replace('const_', '')  # Remove 'const_'
    if '%' in x:
        x = x.replace('%', '')  # Remove '%'
    if 'result' in x:
        x = x.replace('%', '')  # Remove '%'
    return x

In [133]:
def parse_steps(x):
  split_x = x.split(" ")
  # print(split_x)
  arg1 = simplify(split_x[3])
  arg2 = simplify(split_x[5])
  # print(arg1, arg2)
  result = simplify(split_x[-1])
  output = {'Operation': split_x[2], 'argument_1': arg1,
                    'argument_2': arg2,'result': result}
  return output

In [134]:
parse_steps(result_df["generated_text"][0]),parse_steps(result_df["label_text"][0])

({'Operation': 'Divide',
  'argument_1': '637',
  'argument_2': '5.',
  'result': '647'},
 {'Operation': 'Divide',
  'argument_1': '637',
  'argument_2': '5.',
  'result': '127.40'})

In [135]:
pd.DataFrame(result_df["generated_text"].apply(lambda x: parse_steps(x)).tolist())

,Operation,argument_1,argument_2,result
0,Divide,637,5.,647
1,Subtract,100,193.5.,193.5
2,Divide,60,243.,11
3,Subtract,536.7,555.3.,5.3
4,Subtract,59.1,922.9.,-9.5
5,Subtract,10815,11503.,-2106
6,Divide,201,167.,0.91
7,Divide,36197,381256.,result:.5838
8,Subtract,339235,338240.,338240
9,Subtract,2255,3693.,-286


In [39]:
pd.DataFrame()

,0
0,None
1,None
2,{'Answer': 'Step 1: Divide 60 by 243. This giv...
3,{'Answer': 'Step 1: Subtract 536.7 from 555.3....
4,{'Answer': 'Step 1: Subtract 59.1 from 922.9. ...
...,...
83,{'Answer': 'Step 1: Subtract 373 from 369. Thi...
84,{'Answer': 'Step 1: Subtract 71 from 189. This...
85,{'Answer': 'Step 1: Subtract 201 from 189. Thi...
86,{'Answer': 'Step 1: Subtract 2504 from 1505. T...
